In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import numpy as np
import re,unicodedata,nltk
from transformers import AutoTokenizer, TFDebertaV2ForSequenceClassification, BertConfig
import keras
import pickle

# gpus = tf.config.list_physical_devices("GPU")
# if gpus:
#     for gpu in gpus:
#         print("Found a GPU with the name:", gpu)
# else:
#     print("Failed to detect a GPU.")


In [3]:
with open('/content/drive/MyDrive/PECE/models/dataset_train_test/dataset_dict.pickle', 'rb') as file:
    dataset_dict = pickle.load(file)

num_classes = dataset_dict['n_class']

In [4]:
model_id = 'PORTULAN/albertina-900m-portuguese-ptbr-encoder-brwac'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFDebertaV2ForSequenceClassification.from_pretrained(model_id,num_labels=num_classes,num_hidden_layers=12,hidden_dropout_prob=0.4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.64M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['deberta.encoder.layer.23.intermediate.dense.bias', 'deberta.encoder.layer.21.attention.self.value_proj.weight', 'deberta.encoder.layer.23.attention.self.key_proj.bias', 'deberta.encoder.layer.22.attention.self.key_proj.bias', 'deberta.encoder.layer.15.attention.output.LayerNorm.weight', 'deberta.encoder.layer.20.attention.output.LayerNorm.bias', 'deberta.encoder.layer.21.output.LayerNorm.bias', 'deberta.encoder.layer.13.attention.output.LayerNorm.weight', 'deberta.encoder.layer.13.attention.self.key_proj.weight', 'deberta.encoder.layer.22.output.dense.weight', 'deberta.encoder.layer.19.attention.self.query_proj.weight', 'deberta.encoder.layer.17.attention.self.key_proj.bias', 'deberta.encoder.layer.17.attention.self.value_proj.bias', 'deberta.encoder.layer.18.attention.output.dense.weight', 'deberta.encoder.layer.14.attention.output.LayerNorm.bias', 'deberta.encode

In [5]:
def applytokenizer(sentences):
  input_ids=[]
  attention_masks=[]

  for sent in sentences:
      bert_inp=tokenizer.encode_plus(sent,add_special_tokens = True,max_length = 64, padding='max_length',return_attention_mask = True)
      input_ids.append(bert_inp['input_ids'])
      attention_masks.append(bert_inp['attention_mask'])

  input_ids=np.asarray(input_ids)
  attention_masks=np.array(attention_masks)
  return [input_ids,attention_masks]


In [6]:
X_train = applytokenizer(dataset_dict['X_train'])
X_test = applytokenizer(dataset_dict['X_test'])
y_train = np.asarray(dataset_dict['y_train'])
y_test = np.asarray(dataset_dict['y_test'])

In [7]:
log_dir='/content/drive/MyDrive/PECE/tensorboard_data/tb_albertina_v4'
model_save_path='/content/drive/MyDrive/PECE/models/albertina_model_v4.h5'

callbacks = [
             keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),
             keras.callbacks.TensorBoard(log_dir=log_dir),
             keras.callbacks.EarlyStopping(monitor="val_loss",min_delta=0.01,patience=50,verbose=0,mode="min")
             ]

print('\nalBERTina Model',model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

Model: "tf_deberta_v2_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deberta (TFDebertaV2MainLa  multiple                  544614912 
 yer)                                                            
                                                                 
 pooler (TFDebertaV2Context  multiple                  2360832   
 Pooler)                                                         
                                                                 
 cls_dropout (TFDebertaV2St  multiple                  0         
 ableDropout)                                                    
                                                                 
 classifier (Dense)          multiple                  26129     
                                                                 
Total params: 547001873 (2.04 GB)
Trainable params: 547001873 (2.04 GB)
Non-trainable param

In [ ]:
history=model.fit(X_train,y_train,batch_size=32,epochs=600,validation_data=(X_test,y_test),callbacks=callbacks)

Epoch 1/600


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


9/9 [==============================] - 101s 5s/step - loss: 2.8881 - accuracy: 0.0607 - val_loss: 2.8295 - val_accuracy: 0.1000
Epoch 2/600
9/9 [==============================] - 11s 1s/step - loss: 2.8322 - accuracy: 0.0786 - val_loss: 2.8245 - val_accuracy: 0.1000
Epoch 3/600
9/9 [==============================] - 11s 1s/step - loss: 2.8442 - accuracy: 0.0929 - val_loss: 2.8130 - val_accuracy: 0.1714
Epoch 4/600
9/9 [==============================] - 11s 1s/step - loss: 2.7889 - accuracy: 0.1036 - val_loss: 2.7859 - val_accuracy: 0.1429
Epoch 5/600
9/9 [==============================] - 5s 593ms/step - loss: 2.7388 - accuracy: 0.0964 - val_loss: 2.7924 - val_accuracy: 0.1143
Epoch 6/600
9/9 [==============================] - 5s 591ms/step - loss: 2.7749 - accuracy: 0.1179 - val_loss: 2.7933 - val_accuracy: 0.1429
Epoch 7/600
9/9 [==============================] - 5s 602ms/step - loss: 2.7843 - accuracy: 0.1429 - val_loss: 2.7993 - val_accuracy: 0.1286
Epoch 8/600
9/9 [===============

In [ ]:
with open('/content/drive/MyDrive/PECE/models/albertina_model_v4', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)
model.config.to_json_file('/content/drive/MyDrive/PECE/models/Config-alBERTina-fine-tuning_v4.json')

In [ ]:
import keras
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('alBERTina Fine-Tuning')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.savefig('/content/drive/MyDrive/PECE/models/alBERTina-fine-tuning.png')
model.config.to_json_file('/content/drive/MyDrive/PECE/models/Config-alBERTina-fine-tuning.json')

In [ ]:



import json

# Convert and write JSON object to file
with open("/content/drive/MyDrive/PECE/resoucers/dict_classes.json", "w") as outfile:
    json.dump(dict_classes, outfile)



In [ ]:
dict_classes = dict([i for i in zip(data['gt'].astype(str).values,data['label'].values)])
labels_dict = [dict_classes[str(i)] for i in [*range(len(list(dict_classes.keys())))]]

In [ ]:
model_id = 'neuralmind/bert-base-portuguese-cased'
model_save_path='/content/drive/MyDrive/PECE/models/bertimbau_model.h5'

tokenizer = AutoTokenizer.from_pretrained(model_id)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

trained_model = TFBertForSequenceClassification.from_pretrained(model_id,num_labels=num_classes)
# trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

preds = trained_model.predict([val_inp,val_mask],batch_size=1)
pred_labels = np.argmax(preds.logits, axis=1)
f1 = f1_score(val_label,pred_labels,average='micro')
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels,target_names=labels_dict))

print('Training and saving built model.....')

#FOCO NO RECALL > 80%
#APOS ISSO PRECISAO

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


55/55 [==============================] - 4s 8ms/step
F1 score 0.9090909090909091
Classification Report
                                     precision    recall  f1-score   support

               CARGO_DO_FUNCIONARIO       1.00      1.00      1.00         5
        DEPARTAMENTO_DO_FUNCIONARIO       1.00      1.00      1.00         4
           INFORMACAO_SOBRE_CLIENTE       1.00      0.75      0.86         4
                  LOCALIZAR_CLIENTE       0.67      1.00      0.80         2
              LOCALIZAR_FUNCIONARIO       1.00      1.00      1.00         4
   LOCALIZAR_PRODUTOS_PELA_PRODUCAO       0.75      1.00      0.86         3
      LOCALIZAR_PRODUTOS_PELO_CUSTO       0.86      1.00      0.92         6
      METRICAS_DE_CUSTO_DE_PRODUTOS       0.50      1.00      0.67         1
   METRICAS_DE_PRODUCAO_DE_PRODUTOS       1.00      1.00      1.00         5
 METRICAS_DE_PRODUTOS_POR_CATEGORIA       1.00      0.25      0.40         4
              METRICAS_DOS_CLIENTES       0.86   

In [ ]:
teste = "Quantos funcionarios tem no setor de marketing?"

In [ ]:
input_ids=[]
attention_masks=[]

bert_inp=tokenizer.encode_plus(teste,add_special_tokens = True,max_length =64, pad_to_max_length = True,return_attention_mask = True)
input_ids.append(bert_inp['input_ids'])
attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:

preds = trained_model.predict([input_ids,attention_masks],batch_size=1)


1/1 [==============================] - 0s 26ms/step


In [ ]:
teste=np.where(preds.logits>=0.50,1,0)

In [ ]:
preds.logits

array([[-1.7778617 , -0.53754526, -0.90348244, -0.8599582 ,  1.6856688 ,
        -0.44356814, -0.70820117, -1.3051971 , -0.49319133, -0.8795275 ,
         0.80512565,  5.828703  , -0.9201504 ,  2.0146732 , -0.72137386]],
      dtype=float32)

In [ ]:
for i,col in enumerate(teste[0]):
  if col == 1:
    print(dict_classes[i],preds.logits[0][i])

KeyError: 4

In [ ]:
pred_labels = np.argmax(preds.logits, axis=1)

In [ ]:
pred_labels

array([11])

In [ ]:
guia = data.label.unique()

In [ ]:
dict_classes[str(pred_labels[0])]

'METRICAS_DO_QUADRO_DE_FUNCIONARIOS'